In [ ]:
%matplotlib inline
from scipy import stats
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import zipline
from zipline.api import symbol, order_target_percent, set_commission, set_slippage, schedule_function, time_rules, date_rules
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage
import pyfolio as pf

In [ ]:
# Model Settings
initial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

# Comission and Slippage Settings
enable_commission = True
commission_pct = 0.001
enable_slippage = True
slippage_volume_limit = 0.025
slippage_impact = 0.05

In [ ]:
# Momentum Score Function
def momentum_score(ts: pd.Series):
    """
    Input: price time series
    Output: Annualized exponential regression slope multiplied by the R2
    """

    # Make a list of consecutive numbers
    x = np.arange(len(ts))

    # Get logs
    log_ts = np.log(ts)

    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)

    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) -1) * 100

    # Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    
    return score

In [ ]:
# Volatility Function
def volatility(ts: pd.Series):
    return ts.pct_change().rolling(vola_window).std().iloc[-1]

In [ ]:
# Output progress function
def output_progress(context):
    """
    Output some performance numbers during backtest run.
    This code just prints out the past month's performance 
    so that we have something to look at while the backtest runs.
    """

    # Get today's date
    today = zipline.api.get_datetime().date()

    # Calculate percent difference since last month
    perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1

    # Print performance, format as percent with 2 decimals
    print('{} - Last Month Result: {:.2%}'.format(today, perf_pct))

    # Remember today's portfolio value for next month's calculation
    context.last_month = context.portfolio.portfolio_value

In [ ]:
def initialize(context):
    # Set comission and slippage
    if enable_commission:
        comm_model = PerDollar(cost=commission_pct)
    else:
        comm_model = PerDollar(cost=0.0)
    
    set_commission(comm_model)